In [1]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [3]:
#Token as environment variable
['TOKEN'] = '*****************************************************************************************8'

In [4]:
#Verification function
def auth():
    
    return os.getenv('TOKEN')

In [5]:
#Create headers
def create_headers(bearer_token):
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    return headers

In [6]:
#Create query URL
def create_url(keyword,start_date,end_date,max_results):
    
    search_url = "https://api.twitter.com/2/tweets/search/all"

    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields':'author_id,created_at,public_metrics',
                    'expansions':'author_id',
                    'user.fields':'public_metrics',
                    'next_token':{}}
    
    return (search_url, query_params)

In [7]:
#Connect to API endpoint to get response
def connect_to_endpoint(url, headers, params, next_token = None):
    #Use next_token to achieve pagination
    params['next_token'] = next_token
    
    #Output status_code, status_code is 200, which means the transmission is normal
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    
    return response.json()

In [8]:
#Data scraping test

#bearer_token = auth()
#headers = create_headers(bearer_token)

#keyword = '#ETH lang:en'
#start_date ='2020-01-01T00:00:00Z'
#end_date ='2021-01-01T00:00:00Z'
#max_results = '50'
#url = create_url(keyword,start_date,end_date,max_results)

#json_response = connect_to_endpoint(url[0], headers, url[1])

In [9]:
#Integrate tweet data and user data
def merge_response(json_response):
    tweet_data=pd.DataFrame(json_response['data'])
    user_data=pd.DataFrame(json_response['includes']['users'])
    user_data.rename(columns={'public_metrics': 'user_metrics','id':'author_id'},inplace=True)
    raw_data=pd.merge(tweet_data, user_data, how='left', on='author_id')
    return raw_data

In [10]:
#Create a file
#csvFile = open("data.csv", "a", newline="", encoding='utf-8')
#csvWriter = csv.writer(csvFile)

#Create headers
#csvWriter.writerow(['tweet_id', 'created_at', 'text', 'like_count', 'quote_count','reply_count','retweet_count',
#                   'author_id','author_name','user_name','followers_count','following_count','listed_count','tweet_count'])

#Close a file
#csvFile.close()

In [11]:
def append_to_csv(response, fileName):
    
    #Count the number of records
    counter = 0

    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    #Read data cyclically
    for index,tweet in response.iterrows():
        
        tweet_id = tweet['id']
      
        created_at = dateutil.parser.parse(tweet['created_at'])
        
        text = tweet['text']

        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        author_id = tweet['author_id']
        
        author_name=tweet['name']
        
        user_name=tweet['username']
        
        followers_count=tweet['user_metrics']['followers_count']
        following_count=tweet['user_metrics']['following_count']
        listed_count=tweet['user_metrics']['listed_count']
        tweet_count=tweet['user_metrics']['tweet_count']
        
        
        # Create row list
        res = [tweet_id, created_at, text, like_count, quote_count, reply_count, retweet_count,
                    author_id,author_name,user_name,followers_count,following_count,listed_count,tweet_count]
        
        # Add a new line to the file
        csvWriter.writerow(res)
        counter += 1
        
    csvFile.close()

    print("Tweets added from this response: ", counter) 

In [12]:
#Currency type and scraping month
hashtag= '#bitcoin'
month = '1'

#Start time and end time list
start_list =    ['2020-01-01T00:00:00.000Z',
                 '2020-01-02T00:00:00.000Z',
                 '2020-01-03T00:00:00.000Z',
                 '2020-01-04T00:00:00.000Z',
                 '2020-01-05T00:00:00.000Z',
                 '2020-01-06T00:00:00.000Z',
                 '2020-01-07T00:00:00.000Z',
                 '2020-01-08T00:00:00.000Z',
                 '2020-01-09T00:00:00.000Z',
                 '2020-01-10T00:00:00.000Z',
                 '2020-01-11T00:00:00.000Z',
                 '2020-01-12T00:00:00.000Z',
                 '2020-01-13T00:00:00.000Z',
                 '2020-01-14T00:00:00.000Z',
                 '2020-01-15T00:00:00.000Z',
                 '2020-01-16T00:00:00.000Z',
                 '2020-01-17T00:00:00.000Z',
                 '2020-01-18T00:00:00.000Z',
                 '2020-01-19T00:00:00.000Z',
                 '2020-01-20T00:00:00.000Z',
                 '2020-01-21T00:00:00.000Z',
                 '2020-01-22T00:00:00.000Z',
                 '2020-01-23T00:00:00.000Z',
                 '2020-01-24T00:00:00.000Z',
                 '2020-01-25T00:00:00.000Z',
                 '2020-01-26T00:00:00.000Z',
                 '2020-01-27T00:00:00.000Z',
                 '2020-01-28T00:00:00.000Z',
                 '2020-01-29T00:00:00.000Z',
                 '2020-01-30T00:00:00.000Z',
                 '2020-01-31T00:00:00.000Z'
                 ]

end_list =      ['2020-01-01T23:59:59.000Z',
                 '2020-01-02T23:59:59.000Z',
                 '2020-01-03T23:59:59.000Z',
                 '2020-01-04T23:59:59.000Z',
                 '2020-01-05T23:59:59.000Z',
                 '2020-01-06T23:59:59.000Z',
                 '2020-01-07T23:59:59.000Z',
                 '2020-01-08T23:59:59.000Z',
                 '2020-01-09T23:59:59.000Z',
                 '2020-01-10T23:59:59.000Z',
                 '2020-01-11T23:59:59.000Z',
                 '2020-01-12T23:59:59.000Z',
                 '2020-01-13T23:59:59.000Z',
                 '2020-01-14T23:59:59.000Z',
                 '2020-01-15T23:59:59.000Z',
                 '2020-01-16T23:59:59.000Z',
                 '2020-01-17T23:59:59.000Z',
                 '2020-01-18T23:59:59.000Z',
                 '2020-01-19T23:59:59.000Z',
                 '2020-01-20T23:59:59.000Z',
                 '2020-01-21T23:59:59.000Z',
                 '2020-01-22T23:59:59.000Z',
                 '2020-01-23T23:59:59.000Z',
                 '2020-01-24T23:59:59.000Z',
                 '2020-01-25T23:59:59.000Z',
                 '2020-01-26T23:59:59.000Z',
                 '2020-01-27T23:59:59.000Z',
                 '2020-01-28T23:59:59.000Z',
                 '2020-01-29T23:59:59.000Z',
                 '2020-01-30T23:59:59.000Z',
                 '2020-01-31T23:59:59.000Z'
                 ]

#key parameter
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = hashtag + " lang:en "
max_results = 500 #Maximum number of tweets obtained at a time

#Total tweet counter
total_tweets = 0

for i,j in enumerate(start_list):
    
    #Data save path
    fname="E:\\project_data\\tweet_data\\"+hashtag+"\\"+month+"\\"+j[0:10]+".csv"
    
    #Create daily tweet files
    csvFile = open(fname, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['tweet_id', 'created_at', 'text', 'like_count', 'quote_count', 'reply_count', 'retweet_count',
                        'author_id','author_name','user_name','followers_count','following_count','listed_count','tweet_count'])
    csvFile.close()
    
    #Daily key parameters
    count = 0 # Total daily tweets counter
    max_count = 20000 # Maximum daily scraping tweets
    flag = True
    next_token = None
    
    
    while flag:
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']
        
        #next_token exists
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            
            #Data returned by a single request
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                
                response=merge_response(json_response)
                append_to_csv(response, fname)
                
                count += result_count
                total_tweets += result_count
                
                print("Total Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(30)  
                
        # next_token does not exist
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                response=merge_response(json_response)
                append_to_csv(response, fname)
                count += result_count
                total_tweets += result_count
                print("Total Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(30)
            
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevqmnsfyjhuolbjiyjkksf50u5
Start Date:  2020-01-01T00:00:00.000Z
Tweets added from this response:  487
Total Tweets added:  487
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevqmnsfyjhuolbjiyjkksf50u5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevokmjcmbnwvi410w44s1yn2il
Start Date:  2020-01-01T00:00:00.000Z
Tweets added from this response:  487
Total Tweets added:  974
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevokmjcmbnwvi410w44s1yn2il
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevok14lf7k7hzgc9bupfdh5ldp
Start Date:  2020-01-01T00:00:00.000Z
Tweets added from this response:  490
Total Tweets added:  1464
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevok14lf7k7hzgc9bupfdh5ldp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevojq4lcxe3x9v9afoxsrmqsfx
Start Date:  2020-01-01T00:00:00.

-------------------
Token:  b26v89c19zqg8o3fo6yevx1finstniqfj7jv94pig2ut9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevx0u8gh6pppekcgl56h0uxe2l
Start Date:  2020-01-02T00:00:00.000Z
Tweets added from this response:  490
Total Tweets added:  14847
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevx0u8gh6pppekcgl56h0uxe2l
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevuyt5g24nxvsyx0jvl03dkw3h
Start Date:  2020-01-02T00:00:00.000Z
Tweets added from this response:  495
Total Tweets added:  15342
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevuyt5g24nxvsyx0jvl03dkw3h
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevuyid0gv3qqayicqj9as2zaf1
Start Date:  2020-01-02T00:00:00.000Z
Tweets added from this response:  488
Total Tweets added:  15830
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yevuyid0gv3qqayicqj9as2zaf1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yevuyi0v29igeom

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfanrsn5iyb2bbzjdnax57mr4ot
Start Date:  2020-01-03T00:00:00.000Z
Tweets added from this response:  478
Total Tweets added:  28411
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfanrsn5iyb2bbzjdnax57mr4ot
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfanrs9hx8t2ug0hyn70mp35jlp
Start Date:  2020-01-03T00:00:00.000Z
Tweets added from this response:  477
Total Tweets added:  28888
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfanrs9hx8t2ug0hyn70mp35jlp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfanrhim8h7syklbng93u72m9a5
Start Date:  2020-01-03T00:00:00.000Z
Tweets added from this response:  490
Total Tweets added:  29378
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfanrhim8h7syklbng93u72m9a5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfanr6t7w7t1wjveebwax2gfwcd
Start Date:  2020-01-03T00:00:00.000Z
Tweets added from this 

-------------------
Token:  b26v89c19zqg8o3fo6yfawap2fb1ijti64hboinewbhbx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfawa3nn1o6rvsviof40e3saykd
Start Date:  2020-01-04T00:00:00.000Z
Tweets added from this response:  485
Total Tweets added:  42756
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfawa3nn1o6rvsviof40e3saykd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfaw9sp616z3ki9hs8518w68pdp
Start Date:  2020-01-04T00:00:00.000Z
Tweets added from this response:  468
Total Tweets added:  43224
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfaw9sp616z3ki9hs8518w68pdp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfaw9hs5icfth4mcbzv2farelq5
Start Date:  2020-01-04T00:00:00.000Z
Tweets added from this response:  490
Total Tweets added:  43714
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfaw9hs5icfth4mcbzv2farelq5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfaw96y8d4t85oj

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfb4riatdhzir5enf4h8bcc5zst
Start Date:  2020-01-05T00:00:00.000Z
Tweets added from this response:  484
Total Tweets added:  56241
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfb4riatdhzir5enf4h8bcc5zst
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfb2ph6ar5j9uyc138c3gw70sql
Start Date:  2020-01-05T00:00:00.000Z
Tweets added from this response:  486
Total Tweets added:  56727
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfb2ph6ar5j9uyc138c3gw70sql
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfb2p6dvdmncpt94cz0n6i261z1
Start Date:  2020-01-05T00:00:00.000Z
Tweets added from this response:  479
Total Tweets added:  57206
-------------------
-------------------
Token:  b26v89c19zqg8o3fo6yfb2p6dvdmncpt94cz0n6i261z1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo6yfb2ovmz9qom6yy1w06tu8k65x9
Start Date:  2020-01-05T00:00:00.000Z
Tweets added from this 

...

-------------------
Token:  b26v89c19zqg8o3fo71ene5jr50u47gw671vi67h7ffr1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71ene591qgx931nvwp7b9xy90nlp
Start Date:  2020-01-28T00:00:00.000Z
Tweets added from this response:  488
Total Tweets added:  406828
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71ene591qgx931nvwp7b9xy90nlp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71ene58plwxg3liogx5vo7u3rlh9
Start Date:  2020-01-28T00:00:00.000Z
Tweets added from this response:  488
Total Tweets added:  407316
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71ene58plwxg3liogx5vo7u3rlh9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71ene4xypsstynnvgogw3qru9cvx
Start Date:  2020-01-28T00:00:00.000Z
Tweets added from this response:  478
Total Tweets added:  407794
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71ene4xypsstynnvgogw3qru9cvx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enc2wvpdmgc

Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enmp1r9c54bd9x5jdpw94gl2m5
Start Date:  2020-01-29T00:00:00.000Z
Tweets added from this response:  486
Total Tweets added:  420839
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71enmp1r9c54bd9x5jdpw94gl2m5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enmor4xiupp3x64xgu0pebjbzx
Start Date:  2020-01-29T00:00:00.000Z
Tweets added from this response:  483
Total Tweets added:  421322
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71enmor4xiupp3x64xgu0pebjbzx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enmoqxceqcu5e9a4yx4qkxi9vh
Start Date:  2020-01-29T00:00:00.000Z
Tweets added from this response:  483
Total Tweets added:  421805
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71enmoqxceqcu5e9a4yx4qkxi9vh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71enmog7wkretxtszn0f6vtxlo59
Start Date:  2020-01-29T00:00:00.000Z
Tweets added from th

-------------------
Token:  b26v89c19zqg8o3fo71env9gaq28aejiio8xy8ehrsq65
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71env95ibr2y910z0w20lo9xfcl9
Start Date:  2020-01-30T00:00:00.000Z
Tweets added from this response:  480
Total Tweets added:  435091
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71env95ibr2y910z0w20lo9xfcl9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71env8upuvmz64hxgfdnbjbc9dvh
Start Date:  2020-01-30T00:00:00.000Z
Tweets added from this response:  474
Total Tweets added:  435565
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71env8upuvmz64hxgfdnbjbc9dvh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71env8jvxybgyg0227tpwrw5nofx
Start Date:  2020-01-30T00:00:00.000Z
Tweets added from this response:  479
Total Tweets added:  436044
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71env8jvxybgyg0227tpwrw5nofx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71env896ijfu4

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o2qet1pcoy74aycu4bgt72pp
Start Date:  2020-01-31T00:00:00.000Z
Tweets added from this response:  465
Total Tweets added:  448802
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71f2o2qet1pcoy74aycu4bgt72pp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o2531eakvks5ikig3tgjd9j1
Start Date:  2020-01-31T00:00:00.000Z
Tweets added from this response:  462
Total Tweets added:  449264
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71f2o2531eakvks5ikig3tgjd9j1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o1u63n5glp1n1uf5bhhlzipp
Start Date:  2020-01-31T00:00:00.000Z
Tweets added from this response:  471
Total Tweets added:  449735
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71f2o1u63n5glp1n1uf5bhhlzipp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2o1jc5nc1uy763henejk1nzp9
Start Date:  2020-01-31T0

-------------------
Token:  b26v89c19zqg8o3fo71f2hnnn18ioxnxedvd7tjdbzvct
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2hn2csufzx1iu56k14w5bfkot
Start Date:  2020-01-31T00:00:00.000Z
Tweets added from this response:  480
Total Tweets added:  462702
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71f2hn2csufzx1iu56k14w5bfkot
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-31T00:00:00.000Z
Tweets added from this response:  186
Total Tweets added:  462888
-------------------
Total number of results:  462888
